In [121]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [122]:
data_folder = '/Users/cep4u/JingEdward/tunen/data/content_rawdata'
filename = 'spotify_audio_content_db_2M.csv'
file_path = f"{data_folder}/{filename}"

df = pd.read_csv(file_path)

/var/folders/02/1x2mrbmn3kz5lz54txkdr_wm0000gn/T/ipykernel_60963/1069100813.py:5: DtypeWarning: Columns (2,3,5,8,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [123]:
one_hot = pd.get_dummies(df['genre'], prefix='genre').astype(int)

df = pd.concat([df, one_hot], axis=1)

In [124]:
one_hot = pd.get_dummies(df['mode'], prefix='mode').astype(int)

df = pd.concat([df, one_hot], axis=1)

In [125]:
df['loudness_scaled'] = (df['loudness'] - df['loudness'].mean()) / df['loudness'].std()

In [126]:
df['scaled_tempo'] = (df['tempo'] - df['tempo'].min()) / (df['tempo'].max() - df['tempo'].min())

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [128]:
df.dropna(subset=['artist_name', 'track_name'], inplace=True)

In [129]:
df['combined'] = df['artist_name'] + " " + df['track_name']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer().fit(df['combined'])
vectors = vectorizer.transform(df['combined'])

# Train Nearest Neighbors
nn = NearestNeighbors(n_neighbors=1, metric='cosine').fit(vectors)

In [130]:
filename = 'knn_vectorizer.pkl'
with open(filename, 'wb') as file:
    pickle.dump(vectorizer, file)

In [131]:
filename = 'knn_query_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(nn, file)

In [132]:
data_folder = '/Users/cep4u/JingEdward/tunen/data/content_rawdata'
filename = 'spotify_audio_content_db_2M_standardized.csv'
file_path = f"{data_folder}/{filename}"

df.to_csv(file_path)

In [133]:
columns_ = ['danceability','energy', 
            'speechiness', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'loudness_scaled', 'scaled_tempo', 'genre_acoustic',
 'genre_afrobeat',
 'genre_alt-rock',
 'genre_ambient',
 'genre_black-metal',
 'genre_blues',
 'genre_breakbeat',
 'genre_cantopop',
 'genre_chicago-house',
 'genre_chill',
 'genre_classical',
 'genre_club',
 'genre_comedy',
 'genre_country',
 'genre_dance',
 'genre_dancehall',
 'genre_death-metal',
 'genre_deep-house',
 'genre_detroit-techno',
 'genre_disco',
 'genre_drum-and-bass',
 'genre_dub',
 'genre_dubstep',
 'genre_edm',
 'genre_electro',
 'genre_electronic',
 'genre_emo',
 'genre_folk',
 'genre_forro',
 'genre_french',
 'genre_funk',
 'genre_garage',
 'genre_german',
 'genre_gospel',
 'genre_goth',
 'genre_grindcore',
 'genre_groove',
 'genre_guitar',
 'genre_hard-rock',
 'genre_hardcore',
 'genre_hardstyle',
 'genre_heavy-metal',
 'genre_hip-hop',
 'genre_house',
 'genre_indian',
 'genre_indie-pop',
 'genre_industrial',
 'genre_jazz',
 'genre_k-pop',
 'genre_metal',
 'genre_metalcore',
 'genre_minimal-techno',
 'genre_new-age',
 'genre_opera',
 'genre_party',
 'genre_piano',
 'genre_pop',
 'genre_pop-film',
 'genre_power-pop',
 'genre_progressive-house',
 'genre_psych-rock',
 'genre_punk',
 'genre_punk-rock',
 'genre_rock',
 'genre_rock-n-roll',
 'genre_romance',
 'genre_sad',
 'genre_salsa',
 'genre_samba',
 'genre_sertanejo',
 'genre_show-tunes',
 'genre_singer-songwriter',
 'genre_ska',
 'genre_sleep',
 'genre_songwriter',
 'genre_soul',
 'genre_spanish',
 'genre_swedish',
 'genre_tango',
 'genre_techno',
 'genre_trance',
 'genre_trip-hop'
           ]

In [134]:
data = df[columns_]

In [135]:
knn_model = NearestNeighbors(algorithm='kd_tree', n_neighbors=200)
knn_model.fit(data)

NearestNeighbors(algorithm='kd_tree', n_neighbors=200)

In [136]:
import pickle

filename = 'knn_200_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(knn_model, file)

In [ ]:
df[df['artist_name'] == 'Djo'].head(20)

In [76]:
query_point = data.iloc[1517578].values.reshape(1, -1)
distances, indices = knn_model.kneighbors(query_point)

/Users/cep4u/mambaforge/envs/melodiik/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [ ]:
for i in indices[0].tolist():
    print(df['popularity'].iloc[i])
    if df['popularity'].iloc[i] < 90.0:
        
        print(i)
        print(df['artist_name'].iloc[i])
        print(df['track_name'].iloc[i])
        print()

In [ ]:
artist = df[df['artist_name'] == 'White Denim']
artist[['popularity']]

In [ ]:
# pseudocode

# Load trained model
# Type in song and find closest match
# Get songs nearest neighbors IDs
# Lookup IDs in DF, return songs with same (or similar) genre
# Filter songs by popularity, no popular artists
    # Introduce method to mark artist as popular?
# Remove duplicate artists
# Return top 20 songs as playlist

In [ ]:
## Structure

In [137]:
class LoadModel:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model = self.load_model()

    def load_model(self):
        with open(self.model_path, 'rb') as file:
            model = pickle.load(file)
        return model

In [138]:
loader = LoadModel('knn_200_model.pkl')
knn_200_model = loader.model

In [139]:
loader = LoadModel('knn_query_model.pkl')
knn_query_model = loader.model

In [140]:
loader = LoadModel('knn_vectorizer.pkl')
knn_vectorizer = loader.model

In [141]:
from fuzzywuzzy import fuzz, process

class RetrieveSongs:
    def __init__(self, data_df, x_data, knn_200_model, knn_query_model, knn_vectorizer):
        self.data_df = data_df
        self.knn_200_model = knn_200_model
        self.knn_query_model = knn_query_model
        self.knn_vectorizer = knn_vectorizer
        self.x_data = x_data

    def search_song(self, query):
        query_vector = self.knn_vectorizer.transform([query])
        distance, index = self.knn_query_model.kneighbors(query_vector)
        return self.data_df.iloc[index[0]]

    def get_nearest_neighbors_ids(self, song_id):
        query_point = self.x_data.iloc[song_id].values.reshape(1, -1)
        distances, indices = self.knn_200_model.kneighbors(query_point)
        return indices

In [142]:
retriever = RetrieveSongs(df, data, knn_200_model, knn_query_model, knn_vectorizer)
song = retriever.search_song("Chateau DJO")
song_id = song.index.tolist()[0]
query_point = data.iloc[song_id].values.reshape(1, -1)
nn_200_indices = retriever.get_nearest_neighbors_ids(song_id)

/Users/cep4u/mambaforge/envs/melodiik/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [148]:
df.loc[nn_200_indices.tolist()[0]].head()

,track_id,track_name,album,album_id,artist_name,artist_ids,track_number,disc_number,explicit,danceability,...,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,mode_0,mode_1,loudness_scaled,scaled_tempo,combined
1517578,3vjs2MDHoF9xhylNg6Y9un,Chateau (Feel Alright),NaN,NaN,Djo,NaN,NaN,NaN,NaN,0.641,...,0,0,0,0,0,0,1,-0.195453,0.539511,Djo Chateau (Feel Alright)
1946642,2ctc9nwRgXeeIFaFWTRrRl,No Te Pido Flores,NaN,NaN,Fanny Lu,NaN,NaN,NaN,NaN,0.728,...,0,0,0,0,0,0,1,0.752046,0.415892,Fanny Lu No Te Pido Flores
2029484,02Fzn9E9et1LADcuSeSwxR,Ya Se Que Te Acordaras...(De Mi),NaN,NaN,Los Brios,NaN,NaN,NaN,NaN,0.654,...,0,0,0,0,0,0,1,0.328873,0.391559,Los Brios Ya Se Que Te Acordaras...(De Mi)
2029415,4oL4ShpnvaF8dJrYgcsbPM,Fighting,NaN,NaN,Yellowcard,NaN,NaN,NaN,NaN,0.414,...,0,0,0,0,0,0,1,0.880884,0.719920,Yellowcard Fighting
2072769,3MldzywZOH4Uci6bq9noyJ,What You Got,NaN,NaN,Colby O'Donis,NaN,NaN,NaN,NaN,0.775,...,0,0,0,0,0,0,1,0.572251,0.479909,Colby O'Donis What You Got


In [ ]:
class RankingSongs:
    def __init__(self, songs_df):
        self.songs_df = songs_df

    def filter_by_popularity(self):
        # Logic to filter songs by popularity
        pass

    def mark_artist_as_popular(self, artist_id):
        # Logic to mark an artist as popular
        pass

    def remove_duplicate_artists(self):
        # Logic to remove songs from duplicate artists
        pass


class Playlist:
    def __init__(self):
        self.songs = []

    def add_songs(self, songs):
        # Logic to add songs to the playlist
        self.songs.extend(songs)

    def get_top_songs(self, count=20):
        # Logic to return top N songs, default is 20
        return self.songs[:count]


# Example workflow
model_path = "path/to/your/model"
song_query = "Your input song"

# Load the model
loader = LoadModel(model_path)
model = loader.model

# Get songs
retriever = RetrieveSongs(model, data_df)
song_match = retriever.get_song_match(song_query)
nearest_neighbors = retriever.get_nearest_neighbors_ids(song_match)
songs_of_same_genre = retriever.lookup_by_genre(nearest_neighbors, genre="genre_of_input_song")

# Rank the songs
ranks = RankingSongs(songs_of_same_genre)
ranks.filter_by_popularity()
ranks.remove_duplicate_artists()

# Create playlist
playlist = Playlist()
playlist.add_songs(songs_of_same_genre)
top_songs = playlist.get_top_songs()
